# Huawei Research France

In [100]:
import pandas as pd

In [138]:
sample = 20

test = pd.DataFrame(X_train.source[sample], columns=["current", 
                                          "err_down_bip", 
                                          "err_up_bip", 
                                          "olt_recv", 
                                          "rdown", 
                                          "recv", 
                                          "rup",
                                          "send", "temp", "volt"])
test.head()

,current,err_down_bip,err_up_bip,olt_recv,rdown,recv,rup,send,temp,volt
0,13.0,563.0,25.0,NaN,0.003,-30.459999,0.001,2.32,45.0,3300.0
1,13.0,541.0,48.0,NaN,0.001,-30.459999,0.001,2.25,45.0,3300.0
2,13.0,587.0,68.0,NaN,0.001,-30.459999,0.001,2.35,45.0,3300.0
3,13.0,641.0,79.0,NaN,0.001,-30.459999,0.001,2.23,45.0,3300.0
4,13.0,585.0,96.0,NaN,0.000,-30.459999,0.001,2.19,45.0,3300.0


In [139]:
test.describe()

,current,err_down_bip,err_up_bip,olt_recv,rdown,recv,rup,send,temp,volt
count,672.000000,672.000000,672.000000,0.0,672.000000,672.000000,672.000000,672.00000,672.000000,672.000000
mean,12.660714,733.159241,54.818451,NaN,0.576213,-30.460159,0.010720,2.26110,44.453869,3299.970215
std,0.473820,208.409286,86.458672,NaN,1.941489,0.000160,0.026105,0.05637,0.701891,0.771516
min,12.000000,412.000000,0.000000,NaN,0.000000,-30.459999,0.000000,2.16000,43.000000,3280.000000
25%,12.000000,631.000000,9.000000,NaN,0.001000,-30.459999,0.001000,2.22000,44.000000,3300.000000
50%,13.000000,753.000000,27.000000,NaN,0.001000,-30.459999,0.001000,2.26000,44.000000,3300.000000
75%,13.000000,825.250000,65.250000,NaN,0.102500,-30.459999,0.007000,2.31000,45.000000,3300.000000
max,13.000000,4649.000000,597.000000,NaN,20.476000,-30.459999,0.317000,2.36000,46.000000,3300.000000


In [128]:
import numpy as np
import rampwf as rw

In [129]:
problem = rw.utils.assert_read_problem()

### Les données

In [130]:
X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

Train data
Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples
 Optical Dataset labels composed of
46110 labels of source samples
438 labels of target samples

Test data
Optical Dataset composed of
0 source samples
0 source background samples
17758 target labeled samples
0 target unlabeled samples
47275 target background samples
 Optical Dataset labels composed of
0 labels of source samples
17758 labels of target samples



Les données d'entrée sont tridimensionnelles (échantillon, temps, caractéristiques). Le temps a 672 dimensions (4 fois une heure $\times$ 24 heures $\times$ 7 jours). Il contient des valeurs nan, il doit donc être nettoyé.

In [131]:
X_train.source[6].shape

(672, 10)

### Classification

Vous devez soumettre un extracteur de caractéristiques et un classificateur. La fonction transform de l'extracteur de caractéristiques est exécutée sur chaque donnée d'entrée (cible, source, bkg) et les tableaux résultants sont passés aux fonctions fit et predict du classificateur. L'extracteur de caractéristiques du kit de départ remplace nans par zéro, et aplatit la matrice en **(sample, 6720)**.

In [42]:
%%writefile submissions/source_rf/feature_extractor.py
import numpy as np

class FeatureExtractor:

    def __init__(self):
        pass

    def transform(self, X):
        # Deal with NaNs inplace
        np.nan_to_num(X, copy=False)
        # We flatten the input, originally 3D (sample, time, dim) to
        # 2D (sample, time * dim)
        X = X.reshape(X.shape[0], -1)
        return X


Overwriting submissions/source_rf/feature_extractor.py


In [43]:
%%writefile submissions/source_rf/classifier.py
from sklearn.ensemble import RandomForestClassifier
from utils.dataset import OpticalDataset, OpticalLabels
from lightgbm import LGBMClassifier

import numpy as np

class Classifier:

    def __init__(self):
        self.clf = LGBMClassifier(
            n_estimators=50, 
            max_depth=20, 
            random_state=44, 
            num_leaves=31,
            n_jobs=-1)
        print(self.clf)

    def fit(self, X_source, X_source_bkg, X_target, X_target_unlabeled,
            X_target_bkg, y_source, y_target):
        self.clf.fit(X_source, y_source)

    def predict_proba(self, X_target, X_target_bkg):
        y_proba = self.clf.predict_proba(X_target)
        return y_proba


Overwriting submissions/source_rf/classifier.py


In [67]:
X_train

Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples

In [44]:
trained_workflow = problem.workflow.train_submission('submissions/source_rf', X_train, y_train)
y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)

LGBMClassifier(max_depth=20, n_estimators=50, random_state=44)
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


### Scores

In [45]:
ap    = problem.score_types[0]
rec5  = problem.score_types[1]
rec10 = problem.score_types[2]
rec20 = problem.score_types[3]
acc   = problem.score_types[4]
auc   = problem.score_types[5]

In [46]:
print('ap test score    = {}'.format(ap(y_test.target, y_test_pred[:,1])))
print('rec5 test score  = {}'.format(rec5(y_test.target, y_test_pred[:,1])))
print('rec10 test score = {}'.format(rec10(y_test.target, y_test_pred[:,1])))
print('rec20 test score = {}'.format(rec20(y_test.target, y_test_pred[:,1])))
print('acc test score   = {}'.format(acc(y_test.target, y_test_pred.argmax(axis=1))))
print('auc test score   = {}'.format(auc(y_test.target, y_test_pred[:,1])))

ap test score    = 0.1838770011572445
rec5 test score  = 0.08674804121255875
rec10 test score = 0.16564951837062836
rec20 test score = 0.32083696126937866
acc test score   = 0.823234598490821
auc test score   = 0.6293792968994895


### Cross validation

Nous utilisons une validation croisée dix fois (stratifiée lorsque les étiquettes sont disponibles) pour tous les ensembles de données. Dans chaque split, 20% des instances sont dans l'ensemble de validation, à l'exception des données cibles étiquetées qui servent principalement à la validation (pour obtenir une estimation non biaisée des scores de test, évalués entièrement sur des échantillons cibles étiquetés). Nous plaçons vingt points cibles étiquetés dans les splits d'entraînement. La raison en est que lorsque nous étendons nos services à large bande à la ville B, nous pouvons obtenir rapidement un petit ensemble de données étiquetées, mais nous aimerions déployer notre détecteur de défaillance sans attendre deux mois pour recueillir des données comparables à celles de la ville A.

Le schéma de validation croisée (voir `problem.get_cv`) est implémenté dans la classe `TLShuffleSplit` de `external_imports.utils.cv.py`, si vous voulez y regarder de plus près.

Vous êtes libre de jouer avec la coupure train/test et la validation croisée lors du développement de vos modèles mais sachez que nous utiliserons la même configuration sur le serveur officiel que celle du kit RAMP (sur un ensemble différent de quatre campagnes qui ne sera pas disponible pour vous).

La cellule suivante passe par les mêmes étapes que le script d'évaluation officiel (`ramp-test`).

In [69]:
splits = problem.get_cv(X_train, y_train)

In [13]:
splits = problem.get_cv(X_train, y_train)

y_test_preds = []
for fold_i, (train_is, valid_is) in enumerate(splits):
    trained_workflow = problem.workflow.train_submission(
        'submissions/starting_kit', X_train, y_train, train_is)
    X_fold_train = X_train.slice(train_is)
    X_fold_valid = X_train.slice(valid_is)
    
    y_train_pred = problem.workflow.test_submission(trained_workflow, X_fold_train)
    y_valid_pred = problem.workflow.test_submission(trained_workflow, X_fold_valid)
    y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)
    print('-------------------------------------')
    print('training ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(train_is).target, y_train_pred[:,1])))
    print('validation ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(valid_is).target, y_valid_pred[:,1])))
    print('test ap on fold {} = {}'.format(fold_i, ap(y_test.target, y_test_pred[:,1])))
    
    y_test_preds.append(y_test_pred)

-------------------------------------
training ap on fold 0 = 0.30833333333333335
validation ap on fold 0 = 0.2637875964895809
test ap on fold 0 = 0.16218430339780684
-------------------------------------
training ap on fold 1 = 0.21250000000000002
validation ap on fold 1 = 0.2555942077788053
test ap on fold 1 = 0.16361016472786805
-------------------------------------
training ap on fold 2 = 0.2
validation ap on fold 2 = 0.29440601825201235
test ap on fold 2 = 0.1745388926023523
-------------------------------------
training ap on fold 3 = 0.7375
validation ap on fold 3 = 0.28218715512682335
test ap on fold 3 = 0.16904411795376056
-------------------------------------
training ap on fold 4 = 0.21250000000000002
validation ap on fold 4 = 0.24879604051634688
test ap on fold 4 = 0.16172210972525408


KeyboardInterrupt: 

Nous calculons à la fois le score moyen du test et le score de la mise en sac de vos dix modèles. Le classement officiel sera déterminé par le score de test mis en sac (sur des ensembles de données différents de ceux dont vous disposez). Votre score public sera le score de validation mis en sac (le calcul de la moyenne est [légèrement plus compliqué](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/rampwf/utils/combine.py#L56) car nous devons nous occuper correctement des masques de validation croisée). 

In [16]:
bagged_y_pred = np.array(y_test_preds).mean(axis=0)
print('Mean ap score = {}'.format(
    np.mean([ap(y_test.target, y_test_pred[:,1]) for y_test_pred in y_test_preds])))
print('Bagged ap score = {}'.format(
    ap(y_test.target, np.array([y_test_pred for y_test_pred in y_test_preds]).mean(axis=0)[:,1])))

Mean ap score = 0.1662199176814084
Bagged ap score = 0.1688256992369087


## Exemple submissions

Outre le kit de départ, nous vous proposons deux autres exemples de soumissions. L'extracteur de caractéristiques est le même dans les trois. `source_rf` est similaire au kit de départ, mais utilise des arbres plus nombreux et plus profonds, pour obtenir un meilleur score. `target_rf` est une autre soumission extrême qui utilise seulement l'instance d'entraînement de la cible (peu) étiquetée pour apprendre un classificateur. Il a une performance légèrement moins bonne que `source_rf` ce qui signifie que les données sources améliorent le classificateur même si les distributions sources et cibles sont différentes.

### Resultats:
|          | ap             | rec-5         | rec-10         | rec-20         | acc            |  auc           | 
|:---------|:--------------:|:-------------:|:--------------:|:--------------:|:--------------:|:--------------:|   
|source_rf | 0.191 ± 0.0026 | 0.073 ± 0.002 | 0.176 ± 0.0032 | 0.357 ± 0.0075 | 0.84 ± 0.0014  | 0.637 ± 0.0063 | 
|target_rf | 0.163 ± 0.0218 | 0.067 ± 0.0182| 0.138 ± 0.0339 | 0.272 ± 0.0537 | 0.813 ± 0.036  | 0.591 ± 0.0399 | 

La grande question de l'apprentissage par transfert à résoudre est la suivante : **Comment combiner les données cibles à faible biais et à haute variance avec les données sources à faible biais et à haute variance**. D'autres questions auxquelles nous nous attendons à voir des réponses :

1. Peut-on faire un meilleur prétraitement (amputation des données manquantes, utilisation du temps d'une manière plus intelligente) dans l'extracteur de caractéristiques ?
2. Normalement, les données d'arrière-plan (bonnes instances) ne participent pas au scoring, mais elles peuvent informer le classifieur du changement de distribution. Comment utiliser au mieux cette information ?

## Local testing (before submission)

You submission will contain a `feature_extractor.py` implementing a FeatureExtractor class with a `transform` function (no `fit`) and a `classifier.py` implementing a Classifier class with a `fit` and `predict_proba` functions as in the starting kit. You should place it in the `submission/<submission_name>` folder in your RAMP kit folder. To test your submission, go to your RAMP kit folder in the terminal and type
```
ramp-test --submission <submission_name>
```
It will train and test your submission much like we did it above in this notebook, and print the foldwise and summary scores. You can try it also in this notebook:

In [ ]:
!ramp-test --submission target_rf

If you want to have a local leaderboard, use the `--save-output` option when running `ramp-test`, then try `ramp-show leaderboard` with different options. For example:
```
ramp-show leaderboard --mean --metric "['ap','auc']" --step "['valid','test']" --precision 3
```
and
```
ramp-show leaderboard --bagged --metric "['auc']"
```

RAMP also has an experimental hyperopt feature, with random grid search implemented. If you want to use it, type
```
ramp-hyperopt --help
```
and check out the example submission [here](https://github.com/ramp-kits/titanic/tree/hyperopt/submissions/starting_kit_h).